# XAI Neural Net Creation

## setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
currDir = os.path.dirname(os.path.realpath("__file__"))
rootDir = os.path.abspath(os.path.join(currDir, '..'))
sys.path.insert(1, rootDir)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical

from src.data.make_dataset import make_dataset_1, make_dataset_2
from src.utils.file_utils import save_df, save_model

/usr/local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.0.3) doesn't match a supported version!
  RequestsDependencyWarning)


## generate artificial dataset

In [4]:
df1 = make_dataset_1()
df2 = make_dataset_2()
save_df(df1, "dataset1.csv")
save_df(df2, "dataset2.csv")

successfully generated dataset | num_rows: 2600000
successfully generated dataset | num_rows: 504000
df successfully saved | filename: dataset1.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data
df successfully saved | filename: dataset2.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data


In [5]:
df2.describe()
df1.describe()

,mode,ei,to,td,tf,vers
count,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06
mean,1.160000e+01,1.854442e+00,2.677224e+02,7.155500e+01,1.216666e+01,4.500000e+00
std,8.002501e+00,3.186181e+00,1.555065e+03,4.702838e+01,1.125711e+01,2.872282e+00
min,5.000000e+00,3.479820e-01,1.000000e+00,1.000000e-01,1.000000e+00,0.000000e+00
25%,5.000000e+00,9.330000e-01,3.000000e+00,3.430000e+01,3.968246e+00,2.000000e+00
50%,6.500000e+00,1.354955e+00,5.000000e+00,6.850000e+01,9.000000e+00,4.500000e+00
75%,2.100000e+01,1.354955e+00,5.500000e+01,1.027000e+02,1.700000e+01,7.000000e+00
max,2.600000e+01,2.317735e+01,1.440000e+04,2.600000e+02,6.000000e+01,9.000000e+00


In [11]:
from src.utils.file_utils import load_df
df1, df2 = load_df("dataset1.csv"), load_df("dataset2.csv")

## create neural nets

In [19]:
# def build_model():    
#     model = Sequential()
#     model.add(Dense(6, input_dim=5, activation="relu"))#hidden layer
#     model.add(Dense(10, activation="relu"))#hidden layer
#     model.add(Dense(1, activation='sigmoid'))#output layer

#     optimizer = RMSprop(0.001)
#     model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae', 'mape'])
#     return model

def build_model(num_features, num_classes):
    model = Sequential()
    model.add(Dense(32, input_shape=(num_features,), activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', # Cross-entropy
                optimizer='adam', metrics=['accuracy'])
    return model

def build_model2(num_features, num_classes):
    model = Sequential()
    model.add(Dense(5, input_shape=(num_features,), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', # Cross-entropy
                optimizer='adam', metrics=['accuracy'])
    return model

## train

In [7]:
def prepare_df(df, y_column):
    temp_df = df.copy()
    temp_df.sample(frac=1)
    
    y = to_categorical(
        temp_df[y_column].values)
    temp_df.drop(y_column, axis=1, inplace=True)
    features = list(temp_df.columns)
    
    x = temp_df.values
    scaler = StandardScaler()
    scaler.fit(x)
    x = scaler.transform(x)
    
    return x,y,features

x1, y1, features1 = prepare_df(df1, "vers")
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.20)
norm_train1, norm_test1 = pd.DataFrame(x_train1, columns=features1), pd.DataFrame(x_test1, columns=features1)
save_df(norm_train1, "norm_train1.csv")
save_df(norm_train1, "norm_test1.csv")

x2, y2, features2 = prepare_df(df2, "vers")
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.20)
norm_train2, norm_test2 = pd.DataFrame(x_train2, columns=features2), pd.DataFrame(x_test2, columns=features2)
save_df(norm_train2, "norm_train2.csv")
save_df(norm_train2, "norm_test2.csv")

df successfully saved | filename: norm_train1.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data
df successfully saved | filename: norm_test1.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data
df successfully saved | filename: norm_train2.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data
df successfully saved | filename: norm_test2.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data


In [9]:
model1 = build_model(5,10)
history1 = model1.fit(x_train1, y_train1, epochs=10, validation_split=0.2, batch_size=128, verbose=2)

Train on 1664000 samples, validate on 416000 samples
Epoch 1/10
1664000/1664000 - 28s - loss: 0.9348 - accuracy: 0.6559 - val_loss: 0.7022 - val_accuracy: 0.7404
Epoch 2/10
1664000/1664000 - 27s - loss: 0.6329 - accuracy: 0.7692 - val_loss: 0.5996 - val_accuracy: 0.7742
Epoch 3/10
1664000/1664000 - 22s - loss: 0.5729 - accuracy: 0.7850 - val_loss: 0.5690 - val_accuracy: 0.7804
Epoch 4/10
1664000/1664000 - 21s - loss: 0.5489 - accuracy: 0.7893 - val_loss: 0.5455 - val_accuracy: 0.7829
Epoch 5/10
1664000/1664000 - 21s - loss: 0.5363 - accuracy: 0.7907 - val_loss: 0.5334 - val_accuracy: 0.7917
Epoch 6/10
1664000/1664000 - 21s - loss: 0.5278 - accuracy: 0.7919 - val_loss: 0.5313 - val_accuracy: 0.7908
Epoch 7/10
1664000/1664000 - 21s - loss: 0.5229 - accuracy: 0.7920 - val_loss: 0.5363 - val_accuracy: 0.7895
Epoch 8/10
1664000/1664000 - 21s - loss: 0.5177 - accuracy: 0.7927 - val_loss: 0.5096 - val_accuracy: 0.7957
Epoch 9/10
1664000/1664000 - 22s - loss: 0.5150 - accuracy: 0.7929 - val_lo

In [20]:
model1_2 = build_model2(5,10)
history1_2 = model1_2.fit(x_train1, y_train1, epochs=10, validation_split=0.2, batch_size=128, verbose=2)

Train on 1664000 samples, validate on 416000 samples
Epoch 1/10
1664000/1664000 - 22s - loss: 1.4032 - accuracy: 0.4843 - val_loss: 1.1149 - val_accuracy: 0.6191
Epoch 2/10
1664000/1664000 - 23s - loss: 1.0531 - accuracy: 0.6301 - val_loss: 1.0188 - val_accuracy: 0.6388
Epoch 3/10
1664000/1664000 - 22s - loss: 0.9960 - accuracy: 0.6466 - val_loss: 0.9791 - val_accuracy: 0.6665
Epoch 4/10
1664000/1664000 - 24s - loss: 0.9657 - accuracy: 0.6612 - val_loss: 0.9565 - val_accuracy: 0.6560
Epoch 5/10
1664000/1664000 - 23s - loss: 0.9470 - accuracy: 0.6649 - val_loss: 0.9427 - val_accuracy: 0.6671
Epoch 6/10
1664000/1664000 - 21s - loss: 0.9319 - accuracy: 0.6680 - val_loss: 0.9262 - val_accuracy: 0.6682
Epoch 7/10
1664000/1664000 - 21s - loss: 0.9187 - accuracy: 0.6702 - val_loss: 0.9206 - val_accuracy: 0.6623
Epoch 8/10
1664000/1664000 - 26s - loss: 0.9093 - accuracy: 0.6707 - val_loss: 0.9207 - val_accuracy: 0.6503
Epoch 9/10
1664000/1664000 - 26s - loss: 0.9021 - accuracy: 0.6712 - val_lo

In [10]:
model2 = build_model(4,7)
history2 = model2.fit(x_train2, y_train2, epochs=15, validation_split=0.2, batch_size=128, verbose=2)

Train on 322560 samples, validate on 80640 samples
Epoch 1/15
322560/322560 - 6s - loss: 0.6668 - accuracy: 0.7958 - val_loss: 0.2820 - val_accuracy: 0.9435
Epoch 2/15
322560/322560 - 5s - loss: 0.2228 - accuracy: 0.9446 - val_loss: 0.1930 - val_accuracy: 0.9606
Epoch 3/15
322560/322560 - 6s - loss: 0.1733 - accuracy: 0.9515 - val_loss: 0.1662 - val_accuracy: 0.9506
Epoch 4/15
322560/322560 - 4s - loss: 0.1575 - accuracy: 0.9544 - val_loss: 0.1537 - val_accuracy: 0.9590
Epoch 5/15
322560/322560 - 5s - loss: 0.1497 - accuracy: 0.9569 - val_loss: 0.1468 - val_accuracy: 0.9596
Epoch 6/15
322560/322560 - 5s - loss: 0.1461 - accuracy: 0.9578 - val_loss: 0.1554 - val_accuracy: 0.9533
Epoch 7/15
322560/322560 - 6s - loss: 0.1433 - accuracy: 0.9589 - val_loss: 0.1580 - val_accuracy: 0.9399
Epoch 8/15
322560/322560 - 6s - loss: 0.1409 - accuracy: 0.9599 - val_loss: 0.1489 - val_accuracy: 0.9548
Epoch 9/15
322560/322560 - 5s - loss: 0.1395 - accuracy: 0.9605 - val_loss: 0.1379 - val_accuracy: 0.

In [22]:
model2_2 = build_model2(4,7)
history2_2 = model2_2.fit(x_train2, y_train2, epochs=15, validation_split=0.2, batch_size=128, verbose=2)

Train on 322560 samples, validate on 80640 samples
Epoch 1/15
322560/322560 - 6s - loss: 1.0903 - accuracy: 0.6064 - val_loss: 0.7503 - val_accuracy: 0.6977
Epoch 2/15
322560/322560 - 5s - loss: 0.5726 - accuracy: 0.7817 - val_loss: 0.4501 - val_accuracy: 0.8514
Epoch 3/15
322560/322560 - 5s - loss: 0.3828 - accuracy: 0.8857 - val_loss: 0.3344 - val_accuracy: 0.9168
Epoch 4/15
322560/322560 - 6s - loss: 0.2958 - accuracy: 0.9191 - val_loss: 0.2660 - val_accuracy: 0.9184
Epoch 5/15
322560/322560 - 5s - loss: 0.2371 - accuracy: 0.9383 - val_loss: 0.2188 - val_accuracy: 0.9437
Epoch 6/15
322560/322560 - 7s - loss: 0.2004 - accuracy: 0.9490 - val_loss: 0.1893 - val_accuracy: 0.9528
Epoch 7/15
322560/322560 - 4s - loss: 0.1775 - accuracy: 0.9530 - val_loss: 0.1709 - val_accuracy: 0.9529
Epoch 8/15
322560/322560 - 5s - loss: 0.1637 - accuracy: 0.9544 - val_loss: 0.1612 - val_accuracy: 0.9564
Epoch 9/15
322560/322560 - 4s - loss: 0.1559 - accuracy: 0.9554 - val_loss: 0.1546 - val_accuracy: 0.

## test

In [23]:
print(model1.evaluate(x_test1, y_test1))
print(model1_2.evaluate(x_test1, y_test1))
# [0.09895923781607832, 0.85723215]

520000/520000 [==============================] - 16s 30us/sample - loss: 0.5146 - accuracy: 0.7934
[0.5146434716848227, 0.7934327]
520000/520000 [==============================] - 16s 31us/sample - loss: 0.8897 - accuracy: 0.6935
[0.8896572437011279, 0.6934904]


In [24]:
print(model2.evaluate(x_test2, y_test2))
print(model2_2.evaluate(x_test2, y_test2))

100800/100800 [==============================] - 3s 29us/sample - loss: 0.1387 - accuracy: 0.9669
[0.13872719309100556, 0.966875]
100800/100800 [==============================] - 3s 33us/sample - loss: 0.1413 - accuracy: 0.9599s - los - ETA: 0s - loss: 0.1414 - accuracy: 0.95
[0.14134339284477015, 0.95987105]


In [25]:
save_model(model1, "model1.h5")
save_model(model2, "model2.h5")
save_model(model1_2, "model1_2.h5")
save_model(model2_2, "model2_@.h5")

model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model1.h5
model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model2.h5
model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model1_2.h5
model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model2_@.h5
